In [1]:
import os
import numpy as np
import cv2
import imageio.v2 as imageio
import matplotlib.pyplot as plt

In [18]:
# list all the files in the dataset directory
mask_path = "/home/asif/DBMS-II/SkinDetection/Mask"
non_mask_path = "/home/asif/DBMS-II/SkinDetection/NonMask"
mask_list = os.listdir(mask_path) # skin
non_mask_list = os.listdir(non_mask_path) # non skin

In [19]:
# to randomize and shuffel the lists indices
import random
random.seed(5)

In [30]:
number_of_train_images = 50
number_of_test_images = 10
indices = ["%04d" % x for x in range(number_of_train_images)]

In [31]:
TP = []
TN = []
FP = []
FN = []

In [35]:
# we will do crossfold validation
# we will 10 fold and will do 90% for training and 10% for testing

for i in range(2):
    random.shuffle(indices)

    # first train the training set
    n = 256
    skinPixel = np.zeros(shape=(n, n, n))
    nonSkinPixel = np.zeros(shape=(n, n, n))
    skinPixelSum = 0
    nonSkinPixelSum = 0
    skinProbability = np.zeros(shape=(n, n, n))
    nonSkinProbability = np.zeros(shape=(n, n, n))
    learning_ratio = np.zeros(shape=(n, n, n)) 

    for index in range(len(indices)):
        mask_image = cv2.imread("/home/asif/DBMS-II/SkinDetection/Mask/"+str(indices[index])+".bmp")
        unmask_image = cv2.imread("/home/asif/DBMS-II/SkinDetection/NonMask/"+str(indices[index])+".jpg")

        height, width, channel = mask_image.shape

        for x in range(height):
            for y in range(width):
                blue = mask_image[x, y, 0]
                green = mask_image[x, y, 1]
                red = mask_image[x, y, 2]

                if (blue > 250 and green > 250 and red > 250):  
                    blue = unmask_image[x, y, 0]
                    green = unmask_image[x, y, 1]
                    red = unmask_image[x, y, 2]

                    nonSkinPixel[red, green, blue] += 1
                    nonSkinPixelSum += 1
                else:
                    skinPixel[red, green, blue] += 1
                    skinPixelSum += 1

        print("Image ", index, " - DONE!")
    
    # now calculate the probabilities
    for r in range(n):
        for g in range(n):
            for b in range(n):
                skinProbability[r, g, b] = skinPixel[r, g, b]/skinPixelSum
                nonSkinProbability[r, g, b] = nonSkinPixel[r, g, b]/nonSkinPixelSum

                if (nonSkinPixel[r, g, b] != 0):
                    learning_ratio[r, g, b] = skinProbability[r, g, b] / nonSkinProbability[r, g, b]
                else:
                    learning_ratio[r, g, b] = 0.0

    print("Training "+str(i)+" done")

    # now test
    T = 0.4
    true_skin = 0
    true_non_skin = 0
    false_skin = 0
    false_non_skin = 0

    for index in range(500, 511):
        test_non_mask_image_path = "/home/asif/DBMS-II/SkinDetection/NonMask/0"+str(index)+".jpg"
        #test_image = cv2.imread(test_image_path)
        test_non_mask_image = imageio.imread(test_non_mask_image_path)
        height, width, channel = test_non_mask_image.shape 

        test_mask_image_path = "/home/asif/DBMS-II/SkinDetection/Mask/0"+str(index)+".bmp"
        test_mask_image = imageio.imread(test_mask_image_path)

        for x in range(height):
            for y in range(width):
                r=test_non_mask_image[x][y][0]
                g=test_non_mask_image[x][y][1]
                b=test_non_mask_image[x][y][2]
                
                red =test_mask_image[x][y][0]
                green =test_mask_image[x][y][1]
                blue =test_mask_image[x][y][2]

                if abs(learning_ratio[r][g][b])<T:  # non skin
                    if red>250 and green>250 and blue>250:
                        true_non_skin = true_non_skin + 1
                    else:
                        false_non_skin = false_non_skin + 1
                else: # skin
                    if  red<250 and green<250 and blue<250:
                        true_skin = true_skin + 1
                    else:
                        false_skin = false_skin + 1
    

    #print
    print(true_skin, false_skin, true_non_skin, false_non_skin)
    # now store the results in file
    TP.append(true_skin)
    TN.append(true_non_skin)
    FP.append(false_skin)
    FN.append(false_non_skin)

# Now store the results in file
with open('TP.txt', 'w') as filehandle:
    for listitem in TP:
        filehandle.write(f'{listitem}\n')

with open('TN.txt', 'w') as filehandle:
    for listitem in TN:
        filehandle.write(f'{listitem}\n')

with open('FP.txt', 'w') as filehandle:
    for listitem in FP:
        filehandle.write(f'{listitem}\n')

with open('FN.txt', 'w') as filehandle:
    for listitem in FN:
        filehandle.write(f'{listitem}\n')


Image  0  - DONE!
Image  1  - DONE!
Image  2  - DONE!
Image  3  - DONE!
Image  4  - DONE!
Image  5  - DONE!
Image  6  - DONE!
Image  7  - DONE!
Image  8  - DONE!
Image  9  - DONE!
Image  10  - DONE!
Image  11  - DONE!
Image  12  - DONE!
Image  13  - DONE!
Image  14  - DONE!
Image  15  - DONE!
Image  16  - DONE!
Image  17  - DONE!
Image  18  - DONE!
Image  19  - DONE!
Image  20  - DONE!
Image  21  - DONE!
Image  22  - DONE!
Image  23  - DONE!
Image  24  - DONE!
Image  25  - DONE!
Image  26  - DONE!
Image  27  - DONE!
Image  28  - DONE!
Image  29  - DONE!
Image  30  - DONE!
Image  31  - DONE!
Image  32  - DONE!
Image  33  - DONE!
Image  34  - DONE!
Image  35  - DONE!
Image  36  - DONE!
Image  37  - DONE!
Image  38  - DONE!
Image  39  - DONE!
Image  40  - DONE!
Image  41  - DONE!
Image  42  - DONE!
Image  43  - DONE!
Image  44  - DONE!
Image  45  - DONE!
Image  46  - DONE!
Image  47  - DONE!
Image  48  - DONE!
Image  49  - DONE!
Training 0 done


IndexError: index 81 is out of bounds for axis 0 with size 3

In [20]:

print(mask_list)


['0427.bmp', '0108.bmp', '0472.bmp', '0270.bmp', '0304.bmp', '0182.bmp', '0444.bmp', '0052.bmp', '0018.bmp', '0510.bmp', '0430.bmp', '0110.bmp', '0234.bmp', '0190.bmp', '0120.bmp', '0082.bmp', '0249.bmp', '0392.bmp', '0260.bmp', '0410.bmp', '0101.bmp', '0363.bmp', '0299.bmp', '0051.bmp', '0203.bmp', '0492.bmp', '0361.bmp', '0275.bmp', '0337.bmp', '0214.bmp', '0432.bmp', '0321.bmp', '0395.bmp', '0535.bmp', '0146.bmp', '0327.bmp', '0186.bmp', '0507.bmp', '0183.bmp', '0039.bmp', '0358.bmp', '0377.bmp', '0476.bmp', '0123.bmp', '0478.bmp', '0202.bmp', '0445.bmp', '0001.bmp', '0085.bmp', '0528.bmp', '0330.bmp', '0168.bmp', '0367.bmp', '0389.bmp', '0291.bmp', '0060.bmp', '0477.bmp', '0353.bmp', '0208.bmp', '0175.bmp', '0538.bmp', '0317.bmp', '0122.bmp', '0178.bmp', '0311.bmp', '0237.bmp', '0475.bmp', '0518.bmp', '0313.bmp', '0131.bmp', '0352.bmp', '0530.bmp', '0296.bmp', '0134.bmp', '0396.bmp', '0407.bmp', '0474.bmp', '0117.bmp', '0107.bmp', '0387.bmp', '0184.bmp', '0161.bmp', '0457.bmp', '01